In [17]:
# ! pip install tqdm tensorflow_hub xlrd nltk

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import os
import re
from collections import Counter

In [2]:
import tensorflow_hub as hub
import tensorflow as tf

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [4]:
tqdm.pandas()

In [5]:
data = []
dirname = 'data/thoughts/Thoughts_en/'
for fname in tqdm(os.listdir(dirname)):
    if not fname.endswith('.docx'):
        continue
    with open(f"{dirname}/{fname}") as f:
        data +=  [f.read()]

100%|██████████| 438/438 [00:00<00:00, 620.93it/s]


In [6]:
len(data)

437

In [7]:
sentences = [s for d in data for s in d.split('\n')]

In [8]:
sentences = [s.strip() for s in sentences]

In [9]:
sentences = [re.sub('&#39;', '\'', s) for s in sentences]

In [51]:
sentences = [re.sub('…', '...', s) for s in sentences]

In [52]:
sentences = [s for s in sentences if s]

In [53]:
len(sentences)

11775

In [54]:
df = pd.read_excel('data/SUBTLEXusfrequencyabove1.xls')

In [55]:
df['word'] = df['Word'].str.lower()

In [56]:
df = df.set_index('word')

In [57]:
freqs = df['FREQcount']

In [58]:
words = [w for s in sentences for w in nltk.word_tokenize(s.lower())]

In [59]:
len(words)

94614

In [60]:
word_cnt = Counter(words)

In [61]:
word_cnt.most_common(10)

[('i', 3734),
 ('the', 3402),
 ('to', 2853),
 ('a', 2373),
 (',', 2343),
 ('is', 2028),
 ('it', 1939),
 ('?', 1731),
 ('not', 1605),
 ('what', 1601)]

In [62]:
oov = [word for word in words if re.match('[a-z]+', word) and word not in freqs]

In [63]:
oov_cnt = Counter(oov)

In [70]:
pd.DataFrame(oov_cnt.most_common(), columns=['word','freq']).to_csv('data/oov.csv', index=False)

In [109]:
sent_cnt = Counter(sentences)

In [110]:
sent_cnt.most_common(10)

[('Good', 264),
 ('Do not know', 178),
 ('What else?', 178),
 ('What more?', 71),
 ('Sabba', 69),
 ('Maybe', 49),
 ("what's going on?", 41),
 ('Okay', 39),
 ('What about you?', 37),
 ('Not too much', 36)]

In [25]:
# !pip install tensorflow-hub

In [111]:
uniq = list(sent_cnt.keys())

In [112]:
batch_size = 100
batches = []
for i in tqdm(range(len(uniq)//batch_size)):
    embeddings = embed(uniq[i*batch_size:(i+1)*batch_size])
    batches += [embeddings]

    

100%|██████████| 98/98 [00:02<00:00, 48.53it/s]


In [113]:
embeddings = embed(uniq[(i+1)*batch_size:])
batches += [embeddings]

In [114]:
len(batches)

99

In [116]:
embeddings = tf.concat(batches, axis=0)

In [117]:
embeddings = embeddings.numpy()

In [118]:
np.savetxt('data/thoughts/embeddings.csv', embeddings, delimiter='\t')

In [119]:
! wc -l data/thoughts/embeddings.csv

9884 data/thoughts/embeddings.csv


In [56]:
# ! tail -n 1 data/thoughts/embeddings.csv

In [120]:
len(uniq)

9884

In [121]:
data = '\n'.join(uniq) + '\n'

In [122]:
data = re.sub('\t', '    ', data)

In [123]:
with open('data/thoughts/sentences.txt', 'w') as f:
    f.write(data)